In [ ]:
import csv

with open('input_file_wenhu.csv', mode='w') as fw:
    with open('input_file_shiyang.csv', mode='w') as fs:
        with open('input_file_yunkai.csv', mode='w') as fy:
            
            #output_wenhu = output[:5000]
            #fileds = ['url1', 'q1', 'a1', 'url2', 'q2', 'a2', 'url3', 'q3', 'a3', 'url4', 'q4', 'a4', 'url5', 'q5', 'a5']
            #writer = csv.DictWriter(fw, fieldnames=fileds)
            #writer.writeheader()
            #for i in range(0, len(output_wenhu) - 5, 5):
            #    writer.writerow({'url1':output_wenhu[i]['url'], 'q1':output_wenhu[i]['q'], 'a1':output_wenhu[i]['a'],
            #                    'url2':output_wenhu[i+1]['url'], 'q2':output_wenhu[i+1]['q'], 'a2':output_wenhu[i+1]['a'],
            #                    'url3':output_wenhu[i+2]['url'], 'q3':output_wenhu[i+2]['q'], 'a3':output_wenhu[i+2]['a'],
            #                    'url4':output_wenhu[i+3]['url'], 'q4':output_wenhu[i+3]['q'], 'a4':output_wenhu[i+3]['a'],
            #                    'url5':output_wenhu[i+4]['url'], 'q5':output_wenhu[i+4]['q'], 'a5':output_wenhu[i+4]['a']})
            
            num = 10
            
            output_shiyang = output[5000:20000]
            fields = []
            for i in range(1, num+1):
                fields.extend(['url{}'.format(i), 'q{}'.format(i), 'a{}'.format(i)])
            writer = csv.DictWriter(fs, fieldnames=fields)
            writer.writeheader()
            for i in range(0, len(output_shiyang) - num, num):
                elem = {}
                for j in range(1, num+1):
                    elem['url{}'.format(j)] = output_shiyang[i+j-1]['url']
                    elem['q{}'.format(j)] = output_shiyang[i+j-1]['q']
                    elem['a{}'.format(j)] = output_shiyang[i+j-1]['a']
                
                writer.writerow(elem) 
            
            
            output_yunkai = output[20000:]
            fields = []
            for i in range(1, num+1):
                fields.extend(['url{}'.format(i), 'q{}'.format(i), 'a{}'.format(i)])
            writer = csv.DictWriter(fy, fieldnames=fields)
            writer.writeheader()
            for i in range(0, len(output_yunkai) - num, num):
                elem = {}
                for j in range(1, num+1):
                    elem['url{}'.format(j)] = output_yunkai[i+j-1]['url']
                    elem['q{}'.format(j)] = output_yunkai[i+j-1]['q']
                    elem['a{}'.format(j)] = output_yunkai[i+j-1]['a']
                
                writer.writerow(elem) 
                

In [ ]:
def regex(inp):
    inp = re.sub(r"> +", r">", inp)
    inp = re.sub(r" +<", r"<", inp)
    inp = re.sub(r">@", r">", inp)
    inp = re.sub(r"\*", r"", inp)
    inp = re.sub(r"#", r"", inp)
    inp = re.sub(r"‡", r"", inp)
    #inp = re.sub(r".0([^0-9])", r"\1", inp)
    for month, abb in [("January", "Jan"), ("February", "Feb"), ("March", "March"), ("April", "April"), 
                   ("May", "May"), ("June","June") , ("July", "July"), ("August", "Aug"), 
                   ("September", "Sep"), ("October", "Oct"), ("November", "Nov"), ("December", "Dec")]:
        inp = re.sub(r"({})([0-9]+)".format(month), r"\1 \2", inp)
        inp = re.sub(r"({})([0-9]+)".format(abb), r"\1 \2", inp)
    return inp

In [ ]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import jsonlines
import csv
import re
"""
with open('all_html_original.csv', 'w') as fw:
    fields = ['url', 'content']
    writer = csv.DictWriter(fw, fieldnames=fields)
    writer.writeheader()
"""
with open('WikiTableQuestions/wikidata/train.tables.jsonl') as f:
    data = jsonlines.Reader(f)
    for d in data:
        f = '<table class="wikitable"><tr>'
        for h in d['header']:
            f += "<th> {} </th>".format(h)
        f += "</tr>"
        for r in d['rows']:
            f += "<tr>"
            for elem in r:
                f += "<td> {} </td>".format(elem)
            f += "</tr>"
        f += "</table>"
        #writer.writerow({'url': d['id'], "content": f})
        with open('WikiTableQuestions/wikidata/all_html/{}.html'.format(d['id']), 'w') as fw:
            print >> fw, regex(f)

In [ ]:
import re

def regex_equation(line):
    line = re.sub(r"([^+-])([0-9]+)-([0-9]+)-([0-9]+)-([0-9]+)=", r"\1\2+\3+\4+\5=", line)
    line = re.sub(r"([^+-])([0-9]+)-([0-9]+)-([0-9]+)=", r"\1\2+\3+\4=", line)
    line = re.sub(r"([^+-])([0-9]+)-([0-9]+)=", r"\1\2+\3=", line)
    #line = re.sub(r"([^+-])([0-9]+)-([0-9]+)+([0-9]+)=", r"\1\2+\3+\4=", line)
    #line = re.sub(r"([^+-])([0-9]+)-([0-9]+)-([0-9]+)-([0-9]+)=", r"\1+\2+\3+\4=", line)    
    return line

In [ ]:
regex_equation("67-54-69-68=270)")

In [ ]:
with open('/tmp/list.txt') as f:
    for line in f:
        with open('WikiTableQuestions/wikidata/all_html/' + line.strip(), 'r') as f1:
            content = f1.readline().strip()
        with open('WikiTableQuestions/wikidata/all_html/' + line.strip(), 'w') as f1:
            print >> f1, regex_equation(content)
        #print regex_equation(content)

In [ ]:
import json

with open('WikiTableQuestions/wikidata/train_gold.json') as f:
    raw_output = json.load(f)
    output = ["/".join([str(__) for __ in _]) for _ in raw_output]

with open('WikiTableQuestions/wikidata/all_training.tsv', 'w') as fw:
    print >> fw, "id\tutterance\tcontext\ttargetValue"
    with open('WikiTableQuestions/wikidata/train.jsonl') as f:
        data = jsonlines.Reader(f)
        idx = 1
        for d, o in zip(data, output):
            print >> fw, "{}\t{}\t{}\t{}".format(idx, d['question'].replace('\t', ''), d['table_id'], o)
            idx += 1

In [ ]:
import os
import pandas

files = pandas.read_table('WikiTableQuestions/wikidata/all_training.tsv', delimiter="\t")

output = []
for f in os.listdir('WikiTableQuestions/wikidata/all_html/'):
    if "html" in f:
        numbering = f.split('.')[0]
        results = files[files.context == numbering]
        
        for q, a in zip(results.utterance, results.targetValue):
            tmp = {}
            tmp["url"] = 'https://raw.githubusercontent.com/wenhuchen/Interface/master/WikiTableQuestions/wikidata/all_html/{}.html'.format(numbering)
            tmp["q"] = q
            tmp["a"] = a
            output.append(tmp)

In [ ]:
with open('WikiTableQuestions/data/all_training_new.tsv', 'w') as fw:
    with open('WikiTableQuestions/data/all_training.tsv') as f:
        for line in f:
            if "csv" in line:
                try:
                    t1, t2, t3, t4 = line.strip().split('\t')
                except Exception:
                    print line.strip()
                    continue
                    
                try:
                    f1, f2, f3 = t3.split('/')
                except Exception:
                    print t3.strip()
                    continue
                    
                t3 = f2.split('-')[0] + "-" + f3
                new_line = "\t".join([t1, t2, t3, t4])
                print >> fw, new_line
            else:
                print >> fw, line.strip()

In [ ]:
import pandas

result = pandas.read_table('WikiTableQuestions/wikidata/all_training.tsv')

new_result = result[~result.targetValue.isin(['None', 'n/a', ])]

new_result.to_csv('WikiTableQuestions/wikidata/all_training_new.tsv', sep='\t')

In [ ]:
for r in result['Answer.d1']:
    print r

In [ ]:
import json
import csv
import pandas

result = pandas.read_csv('results_v2.csv')
filtered = result[result.AssignmentStatus=="Approved"]

items = []
for i, r in filtered.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for i in range(1, num+1):
        if r['Input.a{}'.format(i)] not in ['None', 'n/a'] and " par " not in r['Answer.d{}'.format(i)].lower():
            items.append((r['Input.url{}'.format(i)], r['Input.q{}'.format(i)], r['Input.a{}'.format(i)], r['Answer.d{}'.format(i)]))

with open('v2_results.json', 'w') as f:
    json.dump(items, f, indent=2)
    
with open('v2_rewrite_input.csv', 'w') as f:
    #json.dump(items, f, indent=2)
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
        
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(items) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = items[i + j][0]
            elem["s{}".format(j + 1)] = items[i + j][3]
            
        writer.writerow(elem)

In [ ]:
import json
import pandas

with open('v2_results.json', 'r') as f:
    data = json.load(f)

keys = [_[1] for _ in data]
result = pandas.read_csv('results_v2_new.csv')
filtered = result[result.AssignmentStatus=="Approved"]

items = []
for i, r in filtered.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
        
    for i in range(1, num+1):
        if r['Input.a{}'.format(i)] not in ['None', 'n/a'] and " par " not in r['Answer.d{}'.format(i)].lower() and r['Input.q{}'.format(i)] not in keys:
            items.append((r['Input.url{}'.format(i)], r['Input.q{}'.format(i)], r['Input.a{}'.format(i)], r['Answer.d{}'.format(i)]))

In [ ]:
import csv

result = pandas.read_csv('results_v3.csv')

filtered = result[result.AssignmentStatus=="Approved"]

new_items = []
for i, r in filtered.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for i in range(1, num + 1):
        if r['Input.a{}'.format(i)] not in ['None', 'n/a'] and " par " not in str(r['Answer.d{}'.format(i)]).lower():
            new_items.append((r['Input.url{}'.format(i)], r['Input.q{}'.format(i)], r['Input.a{}'.format(i)], r['Answer.d{}'.format(i)]))

new_items = new_items + items

with open('v3_results.json', 'w') as f:
    json.dump(new_items, f, indent=2)
    
num = 5
with open('v3_rewrite_input.csv', 'w') as f:
    #json.dump(items, f, indent=2)
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
        
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(new_items) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = new_items[i + j][0]
            elem["s{}".format(j + 1)] = new_items[i + j][3]
            
        writer.writerow(elem)

In [ ]:
print len(new_items)

In [ ]:
s = ""
for month, abb in [("January", "Jan"), ("February", "Feb"), ("March", "March"), ("April", "April"), 
                   ("May", "May"), ("June","June") , ("July", "July"), ("August", "Aug"), 
                   ("September", "Sep"), ("October", "Oct"), ("November", "Nov"), ("December", "Dec")]:
    s += ";s/({})([1-9]+)/\\1 \\2/g;s/({}),([1-9]+)//g".format(month, abb)
print s

In [ ]:
import pandas
import simplejson

result = pandas.read_csv('v2_refine.csv')
filtered_result = result[result.AssignmentStatus=="Approved"]

items = []
num = 10
for i, r in filtered_result.iterrows():
    for i in range(1, num+1):
        items.append((r["Input.url{}".format(i)],"-" ,"-" , r["Answer.d{}".format(i)]))

with open('v2_rewrite.json', 'w') as f:    
    simplejson.dump(items, f, encoding='utf-8', ignore_nan=True)

In [ ]:
import os
#import pandas
import re
import csv

def dealwithNum(inp):
    inp = re.sub(r"([^0-9.])(0)([0-9])%", r"\1\2.\3%", inp)
    
    inp = re.sub(r"([^0-9.])([1-9])([0-9])([0-9])%", r"\1\2\3.\4%", inp)
    inp = re.sub(r"([^0-9.])(0)([0-9])([0-9])%", r"\1\2.\3\4%", inp)
    
    inp = re.sub(r"([^0-9.])([1-9])([0-9])([0-9])([0-9])%", r"\1\2\3.\4\5%", inp)
    inp = re.sub(r"([^0-9.])(0)([0-9])([0-9])([0-9])%", r"\1\2.\3\4\5%", inp)
    
    inp = re.sub(r"10000%", r"100.00%", inp)
    inp = re.sub(r"1000%", r"100.0%", inp)
    
    inp = re.sub(r"([^0-9.])([1-9])([0-9])([0-9])([0-9])([0-9])%", r"\1\2\3.\4\5\6%", inp)
    inp = re.sub(r"([^0-9.])(0)([0-9])([0-9])([0-9])([0-9])%", r"\1\2.\3\4\5\6%", inp)
    
    inp = re.sub(r",([0-9])%", r".\1%", inp)
    inp = re.sub(r",([0-9])([0-9])%", r".\1\2%", inp)
    inp = re.sub(r",([0-9])([0-9])([0-9])%", r".\1\2\3%", inp)
    
    
    #inp = re.sub(r"([0-9]),([0-9])", r"\1\2", inp)
    return inp


with open('all_html.csv', 'w') as f:
    fields = ["url", "content"]
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for f in os.listdir('WikiTableQuestions/wikidata/all_html/'):
        with open('WikiTableQuestions/wikidata/all_html/' + f, 'r') as fr:
            string = fr.readline().strip()
        
        writer.writerow({"url": f, "content": dealwithNum(string)})

#inp_s = "<td>123% <td> 0123% <td>012% <td>1233%  <td>12333% <td>10000% <td>01333%"
#dealwithNum(inp_s)

In [ ]:
import pandas

table = pandas.read_csv('all_html.csv')

for i, item in table.iterrows():
    name = item['url']
    content = item['content']
    with open('WikiTableQuestions/wikidata/all_html/{}'.format(name), 'w') as f:
        print >> f, content

In [ ]:
import json
import csv
import pandas

result = pandas.read_csv('results_v4_1.csv')

filtered = result[result.AssignmentStatus=="Approved"]

items = []
for i, r in filtered.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for i in range(1, num+1):
        if r['Input.a{}'.format(i)] not in ['None', 'n/a'] and " par " not in r['Answer.d{}'.format(i)].lower():
            items.append((r['Input.url{}'.format(i)], r['Input.q{}'.format(i)], r['Input.a{}'.format(i)], r['Answer.d{}'.format(i)]))

with open('v4_1_results.json', 'w') as f:
    json.dump(items, f, indent=2)

num = 5
with open('v4_rewrite_input.csv', 'w') as f:
    #json.dump(items, f, indent=2)
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
    
    #fields.extend(["url11", "s11"])
    
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(items) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = items[i + j][0]
            elem["s{}".format(j + 1)] = items[i + j][3]
        
        #elem['url11'] = "https://raw.githubusercontent.com/wenhuchen/Interface/master/WikiTableQuestions/wikidata/all_html/2-1236260-1.html"
        #elem['s11'] = "Total is the rank of total"
            
        writer.writerow(elem)

In [ ]:
import pandas
import csv
import json
import simplejson

r1 = pandas.read_csv('partial_refine_v2.csv')
r2 = pandas.read_csv('partial_refine_v3.csv')
r1 = r1[r1.AssignmentStatus=="Approved"]
r2 = r2[r2.AssignmentStatus=="Approved"]

results = {}
finished = []
for i, r in r1.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        html_name = r['Input.url{}'.format(j)].split('/')[-1]
        t = r['Answer.d{}'.format(j)]
        if t and isinstance(t, str) and t.lower() not in ["na", "none", "n/a", "no"]:
            if html_name not in results:
                results[html_name] = {"text": [], "label": []}
            results[html_name]['text'].append(t)
            results[html_name]['label'].append(1)
        finished.append(r['Input.s{}'.format(j)])
        
for i, r in r2.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        html_name = r['Input.url{}'.format(j)].split('/')[-1]
        t = r['Answer.d{}'.format(j)]
        if t and isinstance(t, str) and t.lower() not in ["na", "none", "n/a", "no"]:
            if html_name not in results:
                results[html_name] = {"text": [], "label": []}        
            results[html_name]['text'].append(r['Answer.d{}'.format(j)])
            results[html_name]['label'].append(1)
        finished.append(r['Input.s{}'.format(j)])

r1 = pandas.read_csv('partial_neg_v2.csv')
r2 = pandas.read_csv('partial_neg_v3.csv')
r1 = r1[r1.AssignmentStatus=="Approved"]
r2 = r2[r2.AssignmentStatus=="Approved"]

for i, r in r1.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        t = r['Answer.d{}'.format(j)]
        if t and isinstance(t, str) and t.lower() not in ["na", "none", "n/a", "no"]:
            html_name = r['Input.url{}'.format(j)].split('/')[-1]
            if html_name not in results:
                results[html_name] = {"text": [], "label": []}
            if r['Answer.d{}'.format(j)] not in results[html_name]['text']:
                results[html_name]['text'].append(r['Answer.d{}'.format(j)])
                results[html_name]['label'].append(-1)

for i, r in r2.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        t = r['Answer.d{}'.format(j)]
        if t and isinstance(t, str) and t.lower() not in ["na", "none", "n/a", "no"]:
            html_name = r['Input.url{}'.format(j)].split('/')[-1]
            if html_name not in results:
                results[html_name] = {"text": [], "label": []}        
            results[html_name]['text'].append(r['Answer.d{}'.format(j)])
            results[html_name]['label'].append(-1)
            
with open('READY/prelim.json', 'w') as f:
    simplejson.dump(results, f, encoding='utf-8', ignore_nan=True)

In [ ]:
# Prepare results
inp_v2 = pandas.read_csv('v2_rewrite_input.csv')
inp_v3 = pandas.read_csv('v3_rewrite_input.csv')

not_finished = []
done = 0
for i, r in inp_v2.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        if r['s{}'.format(j)] not in finished:
            not_finished.append((r['url{}'.format(j)], r['s{}'.format(j)]))
        else:
            done += 1
        
for i, r in inp_v3.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        if r['s{}'.format(j)] not in finished:
            not_finished.append((r['url{}'.format(j)], r['s{}'.format(j)]))
        else:
            done += 1

num = 5
with open('v23_left_rewrite_input.csv', 'w') as f:
    #json.dump(items, f, indent=2)
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
    
    #fields.extend(["url11", "s11"])
    
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(not_finished) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = not_finished[i + j][0]
            elem["s{}".format(j + 1)] = not_finished[i + j][1]
        writer.writerow(elem)

In [ ]:
result = pandas.read_csv("input_file_yunkai.csv")
keys = set()
for i, r in result.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        keys.add(r["url{}".format(j)])

In [ ]:
result = pandas.read_csv("v4_rewrite_input.csv")
#keys = set()
for i, r in result.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        if r["url{}".format(j)] in keys:
            keys.remove(r["url{}".format(j)])
unseen_tables = list(keys)

In [ ]:
with open('READY/prelim.json') as f:
    data = json.load(f)

pos = []
neg = []
pos_length = 0
neg_length = 0
for k in data:
    text = data[k]['text']
    label = data[k]['label']
    for t, l in zip(text, label):
        if l == 1:
            pos.append(t)
            pos_length += len(t.split())
        else:
            neg.append(t)
            neg_length += len(t.split())

print len(pos), len(neg)
print (pos_length + 0.0) / len(pos)
print (neg_length + 0.0) / len(neg)

In [ ]:
from collections import Counter

a = Counter()
a.update([1,2,3])
a.update([1,3,4])

print a

In [ ]:
import pandas
import csv

r1 = pandas.read_csv('/Users/wenhuchen/Downloads/Batch_3580679_batch_results.csv')
r2 = pandas.read_csv('/Users/wenhuchen/Downloads/Batch_3584639_batch_results.csv')
r1 = r1[r1.AssignmentStatus=="Approved"]
r2 = r2[r2.AssignmentStatus=="Approved"]

r3 = pandas.concat([r1, r2])
finished = []

r1_inp = pandas.read_csv('v23_left_rewrite_input.csv')
r2_inp = pandas.read_csv('v4_rewrite_input.csv')
r3_inp = pandas.concat([r1_inp, r2_inp])

for i, r in r3.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        #if r["url{}".format(j)] in keys:
        finished.append(r["Input.s{}".format(j)])

print len(finished)
unfinished = []
done = 0
finished_old = []
for i,r in r3_inp.iterrows():
    if "url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        #if r["url{}".format(j)] in keys:
        if r["s{}".format(j)]  not in finished:
            unfinished.append((r["url{}".format(j)], r["s{}".format(j)]))
        else:
            finished_old.append(r["s{}".format(j)])
            done += 1

print done
with open('v234_rest_rewrite_input.csv', 'w') as f:
    #json.dump(items, f, indent=2)
    num = 5
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
        
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(unfinished) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = unfinished[i + j][0]
            elem["s{}".format(j + 1)] = unfinished[i + j][1]
            
        writer.writerow(elem)

In [ ]:
r1 = pandas.read_csv('/Users/wenhuchen/Downloads/Batch_3583922_batch_results.csv')
r2 = pandas.read_csv('/Users/wenhuchen/Downloads/Batch_3579309_batch_results.csv')
r3 = pandas.read_csv('/Users/wenhuchen/Downloads/Batch_3579785_batch_results.csv')

r1 = r1[r1.AssignmentStatus=="Approved"]
r2 = r2[r2.AssignmentStatus=="Approved"]
r3 = r3[r3.AssignmentStatus=="Approved"]

r4 = pandas.concat([r1, r2, r3])
finished = []


for i, r in r4.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        finished.append(r["Input.s{}".format(j)])
print len(finished)

r1_inp = pandas.read_csv('v2_rewrite_input.csv')
r2_inp = pandas.read_csv('v3_rewrite_input.csv')
r3_inp = pandas.read_csv('v4_rewrite_input.csv')
r4_inp = pandas.concat([r1_inp, r2_inp, r3_inp])
unfinished = []
done = 0
for i,r in r4_inp.iterrows():
    if "url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        #if r["url{}".format(j)] in keys:
        if r["s{}".format(j)]  not in finished:
            unfinished.append((r["url{}".format(j)], r["s{}".format(j)]))
        else:
            done += 1

print done
with open('v234_rest_rewrite_fake_input.csv', 'w') as f:
    num = 5
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
        
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(unfinished) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = unfinished[i + j][0]
            elem["s{}".format(j + 1)] = unfinished[i + j][1]
            
        writer.writerow(elem)

In [14]:
import re

def substitute(string):
    string = string.lower()
    
    string = string.replace(r"\n", "")
    string = string.replace(r"\\", "")
    string = string.replace(r"!$", "")
    string = string.replace(r"#", "")
    string = string.replace('–', '-')
    string = string.replace('−', '-')
    string = string.replace('−', '-')
    string = string.replace('â', '')
    string = string.replace('“', '')
    string = string.replace('€', '')    
    string = string.replace("√", "")
    string = string.replace(r'"', '')
    string = string.replace(r'\.0 ', '')
    string = string.replace(r'¹', '')
    string = string.replace(r'‡', '')
    string = string.replace(r'«', '')
    string = string.replace(r'»', '')
    
    string = string.replace(r"'s", " 's")
    string = string.replace(r"'re", " 're")
    string = string.replace(r"'nt", " not")
    string = string.replace(r'@', ' ')
    string = string.replace(r'`', '')
    string = string.replace(r'•', ' ')
    string = string.replace(r'·', ' ')
    string = string.replace(r'²', ' square')
    string = string.replace(r'\[', '')
    string = string.replace(r'\]', '')
    string = string.replace(r'\{', '')
    string = string.replace(r'\}', '')
    string = string.replace(r'\|', '')
    string = string.replace(r'\^', '')
    string = string.replace(r'”', '')

    string = re.sub(r'\[.+\]', ' ', string)
    string = re.sub(r',([0-9]{3})(?![0-9])', r'\1', string)
    #string = re.sub(r'([0-9]{1-3}),([0-9]{3}),([0-9]{3})([^0-9])', r"\1\2\3\4", string)
    #string = re.sub(r'([0-9]{1-3}),([0-9]{3}),([0-9]{3}),([0-9]{3})([^0-9])', r"\1\2\3\4\5", string)
    #string = re.sub(r'([0-9]{1-3}),([0-9]{3}),([0-9]{3}),([0-9]{3}),([0-9]{3})([^0-9])', r"\1\2\3\4\5\6", string)

    string = re.sub(r'([^,]),', r"\1 ,", string)
    string = re.sub(r',([^,])', r", \1", string)
    string = re.sub(r'([^(])\(', r"\1 (", string)
    string = re.sub(r'\)([^)])', r") \1", string)
    string = re.sub(r'\.\)', r")", string)
    string = re.sub(r'([0-9]+)-([0-9]+)-([0-9]+)-([0-9]+)=', r"\1+\2+\3+\4=", string)
    string = re.sub(r'([0-9]+)-([0-9]+)-([0-9]+)=', r"\1+\2+\3=", string)
    string = re.sub(r'([0-9]+)-([0-9]+)=', r"\1+\2=", string)
    string = re.sub(r"' +([0-9])", r"'\1", string)

    string = re.sub(r"([^ ])\+", r"\1 +", string)
    string = re.sub(r"\+([^ ])", r"+ \1", string)
    string = re.sub(r"([^ ])=", r"\1 =", string)
    string = re.sub(r"=([^ ])", r"= \1", string)

    string = re.sub(r"-([^- ])", r"- \1", string)
    string = re.sub(r"([^- ])-", r"\1 -", string)
    string = re.sub(r"-([^- ])", r"- \1", string)
    string = re.sub(r"/([^ ])", r"/ \1", string)
    string = re.sub(r"([^ ])/", r"\1 /", string)
    string = string.replace(r'()', '')
    
    string = re.sub(r'([^0-9 ]):', r"\1 :", string)
    string = re.sub(r':([^0-9 ])', r": \1", string)
    string = re.sub(r'([0-9])pm', r"\1 pm", string)
    string = re.sub(r'([0-9])am', r"\1 am", string)
    string = re.sub(r'([0-9])rpm', r"\1 rpm", string)
    string = re.sub(r'([0-9])km', r"\1 km", string)
    string = re.sub(r'([0-9])cm', r"\1 cm", string)
    string = re.sub(r'([0-9])m', r"\1 m", string)
    string = re.sub(r'([0-9])mm', r"\1 mm", string)
    string = re.sub(r'([0-9])kg', r"\1 kg", string)
    string = re.sub(r'([0-9])g', r"\1 g", string)
    string = re.sub(r'([0-9])kw', r"\1 kw", string)
    string = re.sub(r'([0-9])kv', r"\1 kv", string)
    string = re.sub(r'([0-9])mph', r"\1 mph", string)
    #string = re.sub(r'([0-9])@', r"\1 @", string)
    #string = re.sub(r'@([0-9])', r"@ \1", string)
    string = re.sub(r'category : articles with hcards', r"", string)
    string = re.sub(r'category : articles with hcard', r"", string)
    string = re.sub(r'category : articles without hcard', r"", string)
    
    string = re.sub(r"\.+$", '', string)
    string = re.sub(r',+$', '', string)    
    string = re.sub(r'\s+', ' ', string)
    string = re.sub(r'\.+','.',string)
    string = re.sub(r',+',',',string)
    string = re.sub(r'^ ', '', string)
    string = re.sub(r' $', '', string)
    string = re.sub('70 - 76 - 68 - 214', '70 + 76 + 68 = 214', string)
    return string

#print substitute("fa\g##ga,/// she got(ff)ff...")
#print substitute('(5) fa:"12-12-13=31')

In [66]:
def blacklist(string):
    black = ['bye', "no chart for", "km (mi)", "lb·ft", "ft (m)", "kg (lb)", 
             "a report of report", "report was report", "is 'report'", "?"]
    for b in black:
        if b in string:
            return True
    return False

with open('data/short_subset.txt') as f:
    limit_length = [_.strip() for _ in f.readlines()]

In [21]:
# encoding=utf8
import sys
import pandas

r1_1 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/refine_0.csv")
r1_2 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/refine_1.csv")

r1 = pandas.concat([r1_1, r1_2])
r1 = r1[r1.AssignmentStatus=="Approved"]

f = open('/tmp/output.txt', 'w')

index = 0
finished = {}
trash = []
for i,r in r1.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        orig_input = r["Answer.d{}".format(j)]
        html_name = r['Input.url{}'.format(j)].split('/')[-1] + '.csv'
        if html_name not in limit_length:
            continue
        if isinstance(orig_input, str) and orig_input.lower() in ["na", "n/a", "no"]:
            #trash.append(html_name)
            print "error"
        if isinstance(orig_input, str) and len(orig_input.split(' ')) > 3 and not blacklist(orig_input):
            replaced_sent = substitute(orig_input)
            if 'hcard' not in replaced_sent:
                print >> f, replaced_sent
                index += 1
                if html_name not in finished:
                    finished[html_name] = [[replaced_sent], [1]]
                else:
                    finished[html_name][0].append(replaced_sent)
                    finished[html_name][1].append(1)
                    
r2_1 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/refine_2.csv")
r2_2 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/refine_3.csv")
r2_3 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/refine_4.csv")

r2 = pandas.concat([r2_1, r2_2, r2_3])
r2 = r2[r2.AssignmentStatus=="Approved"]

for i,r in r2.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        orig_input = r["Answer.d{}".format(j)]
        html_name = r['Input.url{}'.format(j)].split('/')[-1] + '.csv'
        if html_name not in limit_length:
            continue     
        if isinstance(orig_input, str) and orig_input.lower() in ["na", "n/a", "no"]:
            print "error"
        if isinstance(orig_input, str) and len(orig_input.split(' ')) > 3 and not blacklist(orig_input):
            replaced_sent = substitute(orig_input)
            if 'hcard' not in replaced_sent:
                print >> f, replaced_sent
                index += 1
                if html_name not in finished:
                    finished[html_name] = [[replaced_sent], [1]]
                else:
                    finished[html_name][0].append(replaced_sent)
                    finished[html_name][1].append(1)


r3_1 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/fake_0.csv")
r3_2 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/fake_1.csv")
r3_3 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/fake_2.csv")
r3_4 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/fake_3.csv")

r3 = pandas.concat([r3_1, r3_2, r3_3, r3_4])
r3 = r3[r3.AssignmentStatus=="Approved"]

f = open('/tmp/output.txt', 'w')

index = 0
for i,r in r3.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        orig_input = r["Answer.d{}".format(j)]
        html_name = r['Input.url{}'.format(j)].split('/')[-1] + '.csv'
        if html_name not in limit_length:
            continue
        if isinstance(orig_input, str) and orig_input.lower() in ["na", "n/a", "no"]:
            print "error"
        if isinstance(orig_input, str) and len(orig_input.split(' ')) > 3 and not blacklist(orig_input):
            replaced_sent = substitute(orig_input)
            if 'hcard' not in replaced_sent:
                print >> f, replaced_sent
                index += 1
                if html_name not in finished:
                    finished[html_name] = [[replaced_sent], [0]]
                else:
                    finished[html_name][0].append(replaced_sent)
                    finished[html_name][1].append(0)

f.close()
print index
import json
with open("READY/training_all.json", 'w') as f:
    json.dump(finished, f, indent=2)

In [ ]:
print trash
import os
import shutil

print trash
for l in trash + ["2-11916083-12.html", "2-1236260-1.html"]:
    try:
        shutil.move("data/all_csv/" + l + ".csv", "data/trash_csv/" + l + ".csv")
    except Exception:
        print "error, {}".format("data/all_csv/" + l + ".csv", "data/trash_csv/" + l + ".csv")

In [ ]:
from bs4 import BeautifulSoup
import urllib2
import csv

url = open('WikiTableQuestions/wikidata/all_html/2-18178551-5.html').readline().strip()
soup = BeautifulSoup(url, "html.parser")

table = soup.findAll("table", {"class":"wikitable"})[0]
rows = table.findAll("tr")

with open('/tmp/editors.csv', "w") as f:
    writer = csv.writer(f)
    for row in rows:
        csv_row = []
        for cell in row.findAll(["td", "th"]):
            csv_row.append(cell.get_text())
        writer.writerow(csv_row)

In [ ]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')

import csv
import os
from bs4 import BeautifulSoup

for filename in os.listdir('WikiTableQuestions/wikidata/all_html/'):
    if filename.endswith(".html"):
        url = open('WikiTableQuestions/wikidata/all_html/' + filename).readline().strip()
        soup = BeautifulSoup(url, "html.parser")
        table = soup.findAll("table", {"class":"wikitable"})
        if len(table) > 0:
            table = table[0]
            rows = table.findAll("tr")
            with open('WikiTableQuestions/wikidata/all_csv/' + filename + '.csv', 'w') as csvfile:
                spamwriter = csv.writer(csvfile, delimiter='#')
                for row in rows:
                    csv_row = []
                    for cell in row.findAll(["td", "th"]):
                        print cell.get_text()
                        csv_row.append(substitute(cell.get_text()))
                        print substitute(cell.get_text())
                        print ""
                    spamwriter.writerow(csv_row)

In [ ]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import csv
import urllib
import json
import os

with open('READY/training_all.json') as f:
    data = json.load(f)
    
files = data.keys()

unseen = []
with open('data/short_subset.txt') as fs:
    for f in fs:
        f = f.strip()
        if f not in files and f in tiny_mapping:
            unseen.append(f)
with open("data/v5_unseen.json", 'w') as f:
    json.dump(unseen, f)

In [ ]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import json
import urllib
import json
import os 
import csv

with open('data/table_to_page_new.json', 'r') as f:
    tiny_mapping = json.load(f)
    
with open("data/v5_unseen.json", 'r') as f:
    unseen = json.load(f)
    
with open('v5_write_input.csv', 'w') as f:
    fields = ["url1", "wiki1", "topic1"]
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()

    for k in unseen:
        if k in tiny_mapping:
            writer.writerow({"url1": 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k, 
                            "wiki1": tiny_mapping[k][1], "topic1": tiny_mapping[k][0]})
        else:
            writer.writerow({"url1": 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k, "wiki1": "javascript:void(0)", "topic1": "None"})

In [ ]:
with open('/tmp/table_to_page.json', 'r') as f:
    old_tiny_mapping1 = json.load(f)
with open('data/table_to_page.json', 'r') as f:
    old_tiny_mapping2 = json.load(f)
    
new_tiny_mapping = {}
for k, v in old_tiny_mapping2.iteritems():
    new_tiny_mapping[k] = [substitute(old_tiny_mapping1[k].split('/')[-1].replace('_', ' ')), old_tiny_mapping2[k]]

with open('data/table_to_page_new.json', 'w') as f:
    json.dump(new_tiny_mapping, f, indent=2)

In [ ]:
with open('/tmp/g.txt', 'w') as f:
    x = "https://en.wikipedia.org/wiki/2007%E2%80%9308_Scottish_Second_Division"
    print >> f, ["topic", urllib.unquote(x).split('/')[-1]]
    x = 'https://en.wikipedia.org/wiki/Ana_Timoti%C4%87'
    print >> f, {"topic": urllib.unquote(x).split('/')[-1]}

In [ ]:
import pandas

name = files[0]
tab = pandas.read_csv('data/all_csv/' + name + '.csv', delimiter='#')
sent = data[name][0][0]
label = data[name][1][0]

tab

In [ ]:
print sent, label

In [ ]:
from fuzzywuzzy import fuzz

activations = []
for i, t in enumerate(tab.columns):
    for j, v in enumerate(tab[t]):
        if isinstance(v, str) and len(v.split(' ')) > 2:
            if fuzz.partial_ratio(v, sent) > 95:
                print t, j
        else:
            if str(v) in sent:
                print t, j

In [3]:
import pandas
from collections import Counter
import pprint

t = pandas.read_csv("/Users/wenhuchen/Downloads/Batch_3598504_batch_results.csv")
t = t[t.AssignmentStatus=="Approved"]


def transform(string):
    if string == "Problematic":
        return "Neutral"
    else:
        return string

kv_pairs = {}
num = 5
for i, r in t.iterrows():
    for j in range(1, num + 1):
        csv_name = r['Input.url{}'.format(j)].split('/')[-1]
        if (csv_name, r['Input.s{}'.format(j)]) not in kv_pairs:
            kv_pairs[(csv_name, r['Input.s{}'.format(j)])] = [r["Input.o{}".format(j)], r["Answer.A{}".format(j)], "None"]
        else:
            kv_pairs[(csv_name, r['Input.s{}'.format(j)])][2] = r["Answer.A{}".format(j)]

#new_kv_pairs = {}
#for k, v in kv_pairs.iteritems():
#    if "Problematic" not in v:
#        new_kv_pairs[k] = v
        #if v[1] == v[2]:
        #    new_kv_pairs[k][0] = v[1]
            
counter = Counter()
p_c = [] 
for k, v in kv_pairs.iteritems():
    tmp = set(v)
    if len(tmp) == 1:
        p_c.append(1)
    elif len(tmp) == 2:
        p_c.append(1/3.)
    else:
        p_c.append(0)
    
    counter.update(v)

p_e = []
for i, j in counter.items():
    p_e.append(j + 0.0)

p_e = [_ / sum(p_e) for _ in p_e]
p_e = sum(_**2 for _ in p_e)

p_c = sum(p_c) / len(p_c)

kappa = (p_c - p_e) / (1 - p_e)
print "kappa = {}".format(kappa)
print counter
for k, v in new_kv_pairs.iteritems():
    if v[0] == 'Contradictory':
        print k
        print v

kappa = 0.505406867561
Counter({'Entailed': 762, 'Contradictory': 650, 'Problematic': 63, 'Neutral': 40})
('2-14347546-5.html.csv', 'the record for the december under 28 with a game under 26 with less than 38 points and a score of 0 - 1 in ot is 12 - 6 - 6 - 1')
['Contradictory', 'Entailed', 'Neutral']
('2-17162128-3.html.csv', 'the united states is the country when to par is + 11 when the total is more than 151')
['Contradictory', 'Contradictory', 'Entailed']
('1-21781578-2.html.csv', 'the original air date for season 2 was january 13 , 2002')
['Contradictory', 'Contradictory', 'Contradictory']
('1-16387653-1.html.csv', 'during the st. kilda and fitzroy game the crowd size was 12656')
['Contradictory', 'Entailed', 'Entailed']
('1-14319023-2.html.csv', 'in 2009 , the mixed double consists of didit juang indrianto and yayu rahayu and hermansyah is the boys singles')
['Contradictory', 'Contradictory', 'Contradictory']
('2-176524-1.html.csv', 'drummond village is 8.91 in area')
['Contradi

In [ ]:
cleaned_pairs = {}

def decide(s1, s2, s3):
    s2int = {'Entailed': 1, 'Neutral': 0, 'Contradictory': -1}
    avg_score = (s2int[s1] + s2int[s2] + s2int[s3]) // 3.
    if avg_score > 0:
        return "Entailed"
    elif avg_score < 0:
        return "Contradictory"
    else:
        return "Neutral"

for k, v in new_kv_pairs.iteritems():
    if k[0] not in cleaned_pairs:
        cleaned_pairs[k[0]] = [[k[1]], [decide(*v)]]
    else:
        cleaned_pairs[k[0]][0].append(k[1])
        cleaned_pairs[k[0]][1].append(decide(*v))

In [ ]:
import json
with open('READY/cleaned.json', 'w') as f:
    json.dump(cleaned_pairs, f)

In [20]:
import pandas

t = pandas.read_csv('/Users/wenhuchen/Downloads/clean/negative_partial.csv')
t = t[t.AssignmentStatus=="Approved"]

print len(t)
finished = []
for i, row in t.iterrows():
    for j in range(1, 11):
        finished.append(row['Input.s{}'.format(j)])

In [21]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import json
import csv
import random

with open('READY/training_all.json') as f:
    data = json.load(f)

with open('data/table_to_page.json') as f:
    mapping = json.load(f)

dummy = [
    # refuted
    [
        'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/2-18847467-2.html.csv',
        'chicago bears has won the game in september',
        '1982 new orleans saints season',
        'https://en.wikipedia.org/wiki/1982_New_Orleans_Saints_season'
    ],
    # erroneous
    [
        'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/2-18847467-2.html.csv',
        'chicago bears is the best team in the league',
        '1982 new orleans saints season',
        'https://en.wikipedia.org/wiki/1982_New_Orleans_Saints_season'
    ],
    [
        'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/2-18847467-2.html.csv',
        'the ninth week against atlant falcons was a big win with 35 - 6',
        '1982 new orleans saints season',
        'https://en.wikipedia.org/wiki/1982_New_Orleans_Saints_season'
    ],
    [
        'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/2-18847467-2.html.csv',
        'the result against st. lous cardinals was a lost',
        '1982 new orleans saints season',
        'https://en.wikipedia.org/wiki/1982_New_Orleans_Saints_season'
    ],
    [
        'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/2-18847467-2.html.csv',
        'san francisco 49ers was competing on noveber 28 , 1982',
        '1982 new orleans saints season',
        'https://en.wikipedia.org/wiki/1982_New_Orleans_Saints_season'
    ],
        [
        'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/2-18847467-2.html.csv',
        'san francisco is is opponent where week is number',
        '1982 new orleans saints season',
        'https://en.wikipedia.org/wiki/1982_New_Orleans_Saints_season'
    ]
]
    
num = 10
count = 0
with open('verify_inputs_fake.csv', 'w') as fs:
    fields = []
    for i in range(0, num + 1):
        fields.extend(['wiki{}'.format(i), 'url{}'.format(i), 's{}'.format(i), 'topic{}'.format(i)])
    csvwriter = csv.DictWriter(fs, fieldnames=fields)
    csvwriter.writeheader()

    seed = random.choice(range(0, len(dummy)))
    buf = {'wiki0': dummy[seed][3], 'topic0': dummy[seed][2], 'url0': dummy[seed][0], 's0': dummy[seed][1]}
    for k, v in data.iteritems():
        entry = data[k]
        for sent, lab in zip(entry[0], entry[1]):
            if lab == 0:
                cur = len(buf) // 4
                if 'url10' in buf:
                    csvwriter.writerow(buf)
                    seed = random.choice(range(0, len(dummy)))
                    buf = {'wiki0': dummy[seed][3], 'topic0': dummy[seed][2], 'url0': dummy[seed][0], 's0': dummy[seed][1]}
                    cur = 1
                    count += 1
                if sent in finished:
                    continue
                buf['url{}'.format(cur)] = 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k
                buf['s{}'.format(cur)] = sent
                if k in mapping:
                    buf['wiki{}'.format(cur)] = mapping[k][1]
                    buf['topic{}'.format(cur)] = mapping[k][0]
                else:
                    buf['wiki{}'.format(cur)] = "#"
                    buf['topic{}'.format(cur)] = "None"
        if count % 500 == 0:
            print count
        #if count > 30:
        #    break
            

In [ ]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import json
import csv

with open('READY/training_all.json') as f:
    data = json.load(f)

num = 11
count = 0
with open('verify_inputs_fake.csv', 'w') as fs:
    fields = []
    for i in range(0, num):
        fields.extend(['wiki{}'.format(i), 'url{}'.format(i), 's{}'.format(i), 'o{}'.format(i)])
    csvwriter = csv.DictWriter(fs, fieldnames=fields)
    csvwriter.writeheader()

    buf = {}
    for k, v in data.iteritems():
        entry = data[k]
        for sent, lab in zip(entry[0], entry[1]):
            cur = len(buf) // 3 + 1
            if cur > num:
                csvwriter.writerow(buf)
                buf = {}
                cur = 1
                count += 1
            buf['url{}'.format(cur)] = 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k
            buf['s{}'.format(cur)] = sent
            if lab == 1:
                buf['o{}'.format(cur)] = "Entailed"
            else:
                buf['o{}'.format(cur)] = "Contradictory"

In [ ]:
import json

data = []
with open('all_sources/full.json', 'r') as f:
    for line in f:
        data.append(json.loads(line.strip()))
    #mapping[str(d['goldAnnotation']['titleId']) + '-' + str(d['tableId'])] = d['goldAnnotation']['title']

In [ ]:
mapping = {}
for d in data:
    mapping[str(d['pgId']) + '-' + str(d['tableId'])] = d['pgTitle']

In [ ]:
import os
import json

tiny_mapping = {}
for f in os.listdir('data/all_csv/'):
    if f.endswith('.csv'):
        _, pageid, tableid = f.split('.')[0].split('-')
        if pageid + '-' + tableid in mapping:
            tiny_mapping[f] = "https://en.wikipedia.org/wiki/" + "_".join(mapping[pageid + '-' + tableid].split(' '))

with open('data/table_to_page.json', 'w') as f:
    json.dump(tiny_mapping, f, indent=2)

In [ ]:
import json

with open('data/table_to_page.json', 'r') as f:
    tiny_mapping = json.load(f)

print len(tiny_mapping)
print tiny_mapping['1-18974269-1.html.csv']
#print tiny_mapping['1-1007688-1.html.csv']

In [ ]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')

import urllib
import json

with open('data/table_to_page.json', 'r') as f:
    tiny_mapping = json.load(f)
    
new_tiny_mapping = {}
for k, v in tiny_mapping.iteritems():
    new_tiny_mapping[k] = "https://en.wikipedia.org/wiki/" + urllib.quote(v[30:].encode('utf8'))

with open('data/table_to_page_new.json', 'w') as f:
    json.dump(new_tiny_mapping, f, indent=2)

In [ ]:
import pandas

t = pandas.read_csv('v5_write_input.csv')
length = len(t)

t = t.head(3000).tail(2000)
#t = t.head(1000)

t.to_csv('v5_write_input_1000_3000.csv', index=False)

t

In [50]:
import json
import pandas

t = pandas.read_csv("/Users/wenhuchen/Downloads/harvest-v2/first1000.csv")
print len(t)
t = t[t.AssignmentStatus=="Approved"]
print len(t)
index = 0
num = 5
finished = {}
for i,r in t.iterrows():
    html_name = r['Input.url1']
    html_name = html_name.split('/')[-1]
    for j in range(1, num + 1):
        orig_input = r["Answer.d{}".format(j)]
        #if isinstance(orig_input, str) and orig_input.lower() in ["na", "n/a", "no"]:
        #    print "error"
        if isinstance(orig_input, str) and len(orig_input.split(' ')) > 3:
            replaced_sent = substitute(orig_input)
            #replaced_sent = orig_input
            index += 1
            if html_name not in finished:
                finished[html_name] = [[replaced_sent], [1]]
            else:
                finished[html_name][0].append(replaced_sent)
                finished[html_name][1].append(1)

with open("READY/round2_first1000.json",'w') as f:
    json.dump(finished, f, indent=2)

In [54]:
import json
import pandas

t = pandas.read_csv("/Users/wenhuchen/Downloads/harvest-v2/last4000.csv")
print len(t)
t = t[t.AssignmentStatus=="Approved"]
print len(t)
index = 0
num = 5
finished = {}
for i,r in t.iterrows():
    html_name = r['Input.url1']
    html_name = html_name.split('/')[-1]
    for j in range(1, num + 1):
        orig_input = r["Answer.d{}".format(j)]
        #if isinstance(orig_input, str) and orig_input.lower() in ["na", "n/a", "no"]:
        #    print "error"
        if isinstance(orig_input, str) and len(orig_input.split(' ')) > 3:
            replaced_sent = substitute(orig_input)
            #replaced_sent = orig_input
            index += 1
            if html_name not in finished:
                finished[html_name] = [[replaced_sent], [1]]
            else:
                finished[html_name][0].append(replaced_sent)
                finished[html_name][1].append(1)

with open("READY/round2_last4000.json",'w') as f:
    json.dump(finished, f, indent=2)

In [7]:
import json
import pandas

t = pandas.read_csv("/Users/wenhuchen/Downloads/harvest-v2/middle2000.csv")
print len(t)
t = t[t.AssignmentStatus=="Approved"]
print len(t)
index = 0
num = 5
finished = {}
for i,r in t.iterrows():
    html_name = r['Input.url1']
    html_name = html_name.split('/')[-1]
    for j in range(1, num + 1):
        orig_input = r["Answer.d{}".format(j)]
        #if isinstance(orig_input, str) and orig_input.lower() in ["na", "n/a", "no"]:
        #    print "error"
        if isinstance(orig_input, str) and len(orig_input.split(' ')) > 3:
            replaced_sent = substitute(orig_input)
            #replaced_sent = orig_input
            index += 1
            if html_name not in finished:
                finished[html_name] = [[replaced_sent], [1]]
            else:
                finished[html_name][0].append(replaced_sent)
                finished[html_name][1].append(1)

with open("READY/round2_middle2000.json",'w') as f:
    json.dump(finished, f, indent=2)

In [56]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import csv

with open("READY/round2_first1000.json") as f:
    data = json.load(f)

with open("data/table_to_page.json") as f:
    mapping = json.load(f)
    
fields = ['url1', 'wiki1', 'topic1', 's1', 's2', 's3', 's4', 's5']
index = 0
with open("rewrite_fake_first1000.csv", 'w') as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for k, v in data.iteritems():
        for i in range(0, len(v[0]), 5):
            v_s = v[0][i:i+5]
            field = {}
            for j, s in enumerate(v_s):
                field['s{}'.format(j + 1)] = s
            field['url1'] = 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k
            field['wiki1'] = mapping[k][1]
            field['topic1'] = mapping[k][0]
            writer.writerow(field)
            index += 1
        #if index > 20:
        #    break

In [8]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import csv

with open("READY/round2_middle2000.json") as f:
    data = json.load(f)

with open("data/table_to_page.json") as f:
    mapping = json.load(f)
    
fields = ['url1', 'wiki1', 'topic1', 's1', 's2', 's3', 's4', 's5']
index = 0
with open("rewrite_fake_middle2000.csv", 'w') as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for k, v in data.iteritems():
        for i in range(0, len(v[0]), 5):
            v_s = v[0][i:i+5]
            field = {}
            for j, s in enumerate(v_s):
                field['s{}'.format(j + 1)] = s
            field['url1'] = 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k
            field['wiki1'] = mapping[k][1]
            field['topic1'] = mapping[k][0]
            writer.writerow(field)
            index += 1
        #if index > 20:
        #    break

In [65]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import csv

with open("READY/round2_last4000.json") as f:
    data = json.load(f)

files = data.keys()
num = 1473
shiyang_data = files[:num]
wanghong_data = files[num:2*num]
yunkai_data = files[2*num:]
    
with open("data/table_to_page.json") as f:
    mapping = json.load(f)
    
fields = ['url1', 'wiki1', 'topic1', 's1', 's2', 's3', 's4', 's5']

with open("rewrite_fake_shiyang.csv", 'w') as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for k in shiyang_data:
        v = data[k]
        for i in range(0, len(v[0]), 5):
            v_s = v[0][i:i+5]
            field = {}
            for j, s in enumerate(v_s):
                field['s{}'.format(j + 1)] = s
            field['url1'] = 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k
            field['wiki1'] = mapping[k][1]
            field['topic1'] = mapping[k][0]
            writer.writerow(field)
            
with open("rewrite_fake_yunkai.csv", 'w') as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for k in yunkai_data:
        v = data[k]
        for i in range(0, len(v[0]), 5):
            v_s = v[0][i:i+5]
            field = {}
            for j, s in enumerate(v_s):
                field['s{}'.format(j + 1)] = s
            field['url1'] = 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k
            field['wiki1'] = mapping[k][1]
            field['topic1'] = mapping[k][0]
            writer.writerow(field)
            
with open("rewrite_fake_wanghong.csv", 'w') as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for k in wanghong_data:
        v = data[k]
        for i in range(0, len(v[0]), 5):
            v_s = v[0][i:i+5]
            field = {}
            for j, s in enumerate(v_s):
                field['s{}'.format(j + 1)] = s
            field['url1'] = 'https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/' + k
            field['wiki1'] = mapping[k][1]
            field['topic1'] = mapping[k][0]
            writer.writerow(field)


In [7]:
import pandas

t = pandas.read_csv('/Users/wenhuchen/Downloads/harvest-v3/partial_positive.csv')
t = t[t.AssignmentStatus=="Approved"]

mapping = {}
for i, r in t.iterrows():
    for i in range(1, 6):
        name = r['Input.url{}'.format(i)].split('/')[-1]
        if name not in mapping:
            mapping[name] = [[r['Input.s{}'.format(i)]], [r['Answer.A{}'.format(i)]]]
        else:
            mapping[name][0].append(r['Input.s{}'.format(i)])
            mapping[name][1].append(r['Answer.A{}'.format(i)])

In [9]:
import json

with open('READY/verify.json', 'w') as f:
    json.dump(mapping, f, indent=2)

In [71]:
import re
import string

def postprocess(inp, backbone):
    print inp
    new_str = []
    for w in inp.split(' '):
        if w[0] in list(string.lowercase) and w in backbone:
            new_str.append(w.capitalize())
        else:
            new_str.append(w)
    inp = " ".join(new_str)
    
    inp = re.sub(r'([^0-9])\.', r'\1', inp)
    inp = re.sub(r'ʼ', '', inp)
    inp = re.sub(r'ʻ', '', inp)
    #inp = re.sub(r' ?- ?', '-', inp)
    #inp = re.sub(r' ?\+ ?', '+', inp)
    #inp = re.sub(r' ?/ ?', '/', inp)
    #inp = re.sub(r" ?'s", "'s", inp)
    #inp = re.sub(r" ?'nt", "'nt", inp)
    inp = inp + '.'
    return inp

In [62]:
postprocess("fa + faf - fafa / faf he does'nt is n't fa. faf", ['kkk'])

fa + faf - fafa / faf he does'nt is n't fa. faf
fa
+
faf
-
fafa
/
faf
he
does'nt
is
n't
fa.
faf


"fa+faf-fafa/faf he does'nt is n't fa faf."

In [72]:
import pandas
import json

t = pandas.read_csv('/Users/wenhuchen/Downloads/clean/positive.csv')
t = t[t.AssignmentStatus=="Approved"]

print len(t) * 10
num = 10
positive = {}
count = 0
for i, row in t.iterrows():
    for j in range(1, num + 1):
        if row['Answer.A{}'.format(j)] == "Entailed":
            name = row['Input.url{}'.format(j)].split('/')[-1]
            if name in limit_length:
                backbone = []
                with open('data/all_csv/' + name, 'r') as f:
                    lines = []
                    for _ in f.readlines():
                        for w in _.strip().split('#'):
                            lines.extend(w.split(' '))
                    #for line in lines[1:]:
                    backbone.extend(lines)
                count += 1
                #print backbone
                if name in positive:
                    positive[name][0].append(postprocess(row['Input.s{}'.format(j)], backbone))
                    positive[name][1].append(1)
                else:
                    positive[name] = [[postprocess(row['Input.s{}'.format(j)], backbone)], [1]]
    if i // 100 == 1:
        break

print count
with open('READY/cleaned_positive.json', 'w') as f:
    json.dump(positive, f, indent=2)

31350
american spirit team had the best and qual 2 time of 59.073
michel jourdain , jr. 's best and qual 2 time was 58.700
jimmy vasser with the team american spirit team johansson had qual 1 of 59.382 and a qual 2 of 58.861
the qual 1 was 1:00.003 and qual 2 was for 59.822 tiago monteiro
paul tracy with team team player 's had a qual 1 time of 59.804 and a best of time of 58.405
the game that scored 7 - 3 had a record of 38 - 18
the game with 13 lsu as the opponent had a score of 8 - 2
the game that ole mississippi was the opponent to mckean (4 - 1) who loss was played at the regions field
the opponent 23 south carolina has a record of 39 - 19
the may 24 game had a record of 40 - 20
episode with production code 7aff03 was directed by brad turner
in season 19 , episode day 7: 2:00 a.m. - 3:00 a.m. there were 10.34 million viewers
37 - 27 (14 - 10) was the recorded conference of the wac conference
missouri valley is the conference of the wichita state school
the smoke point for 30 g of 

the record of the match was a 10 - 3 (1) score , resulting in a win in round 5 with a time of 5:00 minutes
the opponent , georges st - pierre lost in a (unanimous) decision
canton , ohio was the location for the event , fightfest 2 , which lasted only 3 rounds
the opponent , diego gonzalez , had a time of 0:19
lee doski has a 4 - 2 record
paolo quinteros was on the cai zaragoza team that scored 595 points
both poland and great britain have fc notes
croatia is ranked number 2
piotr hojka and jaroslaw godek are rowers for poland which is ranked number 5
denmark is third place in the rank
sweden has margin of victory of 2 strokes in the 1999 event
when north melbourne is the away team , the crowd was 8662
carlton is the home team , and south melbourne is the away team
essendon is the team with windy hills as their home venue
20000 was the crowd of people who watched richmond play as the away team
in an away game , 8662 people watched north melbourne play
the first time that frank williams

In [47]:
t = pandas.read_csv('/Users/wenhuchen/Downloads/Batch_3625238_batch_results.csv')
t['Reject'] = t['Reject'].astype('str')

for i, row in t.iterrows():
    if row['Input.s0'].startswith('the total attendance for week'):
        print row['Input.s0']
        if row['Answer.A0'] != 'Supported':
            t.set_value(i, 'Reject', "did not pass automatic test")
        else:
            t.set_value(i, 'Reject', "")
    elif row['Input.s0'].startswith('chicago bears has won') or row['Input.s0'].startswith('chicago bears is the best'):
        if row['Answer.A0'] != 'Refuted':
            t.set_value(i, 'Reject', "did not pass automatic test")
        else:
            t.set_value(i, 'Reject', "")
    else:
        if row['Answer.A0'] != 'Erroneous':
            t.set_value(i, 'Reject', "did not pass automatic test")
        else:
            t.set_value(i, 'Reject', "")
    
t.to_csv('/Users/wenhuchen/Downloads/uploading.csv')

In [18]:
import os
import pandas

with open('data/short_subset.txt') as f:
    limit_length = [_.strip() for _ in f.readlines()]

for j, f in enumerate(limit_length):
    print "------------", f, "------------"
    t = pandas.read_csv('data/all_csv/' + f, '#')
    for i in t.columns:
        print i, ":", t.at[0, i]
    if j > 100:
        break

------------ 1-1000181-1.html.csv ------------
state / territory : australian capital territory
text / background colour : blue / white
format : yaa nna
current slogan : act celebration of a century 2013
current series : yil 00a
notes : slogan screenprinted on plate
------------ 1-10006830-1.html.csv ------------
aircraft : robinson r - 22
description : light utility helicopter
max gross weight : 1370 lb (635 kg)
total disk area : 497 ft square (46.2 m square)
max disk loading : 2.6 lb / ft square (14 kg / m square)
------------ 1-10007452-3.html.csv ------------
order year : 1992 - 93
manufacturer : gillig
model : phantom (high floor)
fleet series (quantity) : 444 - 464 (21)
powertrain (engine / transmission) : dd s50egr allison wb - 400r
fuel propulsion : diesel
------------ 1-10015132-1.html.csv ------------
player : quincy acy
no : 4
nationality : united states
position : forward
years in toronto : 2012 - present
school / club team : baylor
------------ 1-10015132-14.html.csv -----

round : 1
choice : 3
overall : 3
player name : jim dunaway
position : defensive tackle
college : mississippi
------------ 1-10361230-1.html.csv ------------
round : 1
choice : 25
overall : 25
player name : john ward
position : offensive tackle
college : oklahoma state
------------ 1-10361453-2.html.csv ------------
game : 1
date : sept. 15
opponent : green bay packers
result : win
vikings points : 32
opponents : 17
record : 1 - 0
attendance : 56267
------------ 1-10361625-1.html.csv ------------
round : 1
choice : 27
overall : 27
player name : tommy kramer
position : quarterback
college : rice
------------ 1-1036189-1.html.csv ------------
headstamp id : vii or viiz
primer annulus color : purple
bullet tip color : none
other features : none
functional type : light ball
------------ 1-1037590-1.html.csv ------------
year : 2002
games : 14
games started : 12
completions : 215
attempts : 429
completion % : 50.1
yards : 2294
yards / attempt : 5.3
touchdowns : 12
interceptions : 16
rating :